# Air Passengers Forecast

### Load Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df = pd.read_csv("AirPassengers.csv")
df.head()

In [ ]:
df['Month'] = pd.to_datetime(df['Month'], infer_datetime_format=True)
df = df.set_index(['Month'])
df.head()

### Visualization

In [ ]:
plt.xlabel('Month')
plt.ylabel('Number of Air Passengers')
plt.plot(df['#Passengers'])

In [ ]:
def test_stationarity(ts):
    rol_mean = ts.rolling(window=52,center=False).mean() 
    rol_std = ts.rolling(window=52,center=False).std()

    orig = plt.plot(ts, label='Original')
    mean = plt.plot(rol_mean, color='red', label='Rolling Mean')
    std = plt.plot(rol_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    test_df = adfuller(ts, autolag='AIC')
    output_df = pd.Series(test_df[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for k, v in test_df[4].items():
        output_df['Critical Value (%s)'%k] = v
    print('Results of Dickey-Fuller Test:')
    print(output_df)

In [ ]:
test_stationarity(df['#Passengers'])

### Preprocess - Transformation

In [ ]:
new_df = df.copy()
new_df['#Passengers'] = df['#Passengers'].apply(lambda x: math.log(x)).diff(1)
new_df = new_df.rolling(2).mean()
new_df.dropna(inplace=True)
test_stationarity(new_df['#Passengers'])

In [ ]:
lag_acf = acf(new_df, nlags=20)
lag_pacf = pacf(new_df, nlags=20, method='ols')

plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(new_df)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(new_df)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')            

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(new_df)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(new_df)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()            

### Model Training + Evaluation

In [ ]:
model = ARIMA(np.log(df), order=(1, 1, 0))  
model = model.fit(disp=-1)
model.summary()

In [ ]:
plt.plot(new_df)
plt.plot(model.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((model.fittedvalues[1:]-new_df['#Passengers'])**2))

### Prediction & Reverse Transformations

In [ ]:
pred = pd.Series(model.fittedvalues, copy=True)
print(pred.head())

pred_cumsum = pred.cumsum()
print(pred_cumsum.head())

pred_log = pd.Series(np.log(df)['#Passengers'].iloc[0], index=np.log(df).index)
pred_log = pred_log.add(pred_cumsum, fill_value=0)
print(pred_log.head())

In [ ]:
y_pred = np.exp(pred_log)
plt.plot(df['#Passengers'])
plt.plot(y_pred)
plt.title('RMSE: %.4f'% np.sqrt(sum((y_pred-df['#Passengers'])**2)/len(df['#Passengers'])))

In [ ]:
model.plot_predict(1, 240)
plt.show()

In [ ]:
'''
Inspiration
1. https://github.com/harishkandan/Air-passengers-time-series-forecasting
'''